# Step 0: Mounting Google Drive and Importing Libraries

In [1]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/grpo-verified-reasoner
!ls

Mounted at /content/drive
/content/drive/MyDrive/grpo-verified-reasoner
data			      LICENSE	 outputs    unsloth_compiled_cache
grpo_trainer_lora_model       models	 README.md  _unsloth_sentencepiece_temp
huggingface_tokenizers_cache  notebooks  src	    wandb


In [ ]:
!pip install -q unsloth

In [3]:
INPUT_MODEL_PATH = "/content/drive/MyDrive/grpo-verified-reasoner/models/qwen3-4b-grpo-merged-f32-final"
OUTPUT_GGUF_DIR = "/content/drive/MyDrive/grpo-verified-reasoner/models/qwen3-4b-grpo-f32-gguf"
os.makedirs(OUTPUT_GGUF_DIR, exist_ok=True)

In [3]:
import os
import torch
from google.colab import userdata
from unsloth import FastLanguageModel
from huggingface_hub import HfApi, create_repo

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


# Step 1: Setting up Environment on Local Disk

In [4]:
%cd /content

/content


In [6]:
!rm -rf llama.cpp

In [7]:
!git clone https://github.com/ggerganov/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 75673, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 75673 (delta 70), reused 26 (delta 26), pack-reused 75550 (from 3)
Receiving objects: 100% (75673/75673), 277.36 MiB | 43.51 MiB/s, done.
Resolving deltas: 100% (54911/54911), done.


In [8]:
%cd llama.cpp

/content/llama.cpp


In [9]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 156.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Step 2: Compiling llama.cpp

In [12]:
# Configure with CMake (Disable CUDA if you are CPU-only, otherwise keep ON)
!cmake -B build -DGGML_CUDA=OFF

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1")
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- Found OpenMP_C: 

In [14]:
# Build the binaries
!cmake --build build --config Release -j 8

[  0%] Building CXX object common/CMakeFiles/build_info.dir/build-info.cpp.o
[  0%] Building C object examples/gguf-hash/CMakeFiles/sha256.dir/deps/sha256/sha256.c.o
[  2%] Building C object examples/gguf-hash/CMakeFiles/xxhash.dir/deps/xxhash/xxhash.c.o
[  2%] Building C object ggml/src/CMakeFiles/ggml-base.dir/ggml.c.o
[  2%] Building CXX object vendor/cpp-httplib/CMakeFiles/cpp-httplib.dir/httplib.cpp.o
[  2%] Building C object examples/gguf-hash/CMakeFiles/sha1.dir/deps/sha1/sha1.c.o
[  3%] Building CXX object tools/mtmd/CMakeFiles/llama-llava-cli.dir/deprecation-warning.cpp.o
[  3%] Building CXX object tools/mtmd/CMakeFiles/llama-gemma3-cli.dir/deprecation-warning.cpp.o
[  3%] Built target build_info
[  3%] Building CXX object ggml/src/CMakeFiles/ggml-base.dir/ggml.cpp.o
[  3%] Built target sha1
[  4%] Building CXX object tools/mtmd/CMakeFiles/llama-minicpmv-cli.dir/deprecation-warning.cpp.o
[  4%] Built target sha256
[  4%] Building CXX object tools/mtmd/CMakeFiles/llama-qwen2vl-

# Step 3: Converting to Uncompressed GGUF (FP16)

In [15]:
!python convert_hf_to_gguf.py {INPUT_MODEL_PATH} --outfile /content/temp_f16.gguf

INFO:hf-to-gguf:Loading model: qwen3-4b-grpo-merged-f32-final
INFO:hf-to-gguf:Model architecture: Qwen3ForCausalLM
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: indexing model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00002-of-00004.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00003-of-00004.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00004-of-00004.safetensors'
INFO:hf-to-gguf:heuristics unable to detect tensor dtype, defaulting to --outtype f16
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:token_embd.weight,         torch.float32 --> F16, shape = {2560, 151936}
INFO:hf-to-gguf:blk.0.attn_norm.weight,    torch.float32 --> F32, shape = {2560}
INFO:hf-to-gguf:blk.0.ffn_down.weight,     torch.float32 --> F16, shape = {9728, 2560}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,     torc

# Step 4: Quantizing to Q4_K_M

In [17]:
!./build/bin/llama-quantize /content/temp_f16.gguf /content/qwen_q4_k_m.gguf q4_k_m

main: build = 7708 (0c3b7a9ef)
main: built with GNU 11.4.0 for Linux x86_64
main: quantizing '/content/temp_f16.gguf' to '/content/qwen_q4_k_m.gguf' as Q4_K_M
llama_model_loader: direct I/O is enabled, disabling mmap
llama_model_loader: loaded meta data with 26 key-value pairs and 398 tensors from /content/temp_f16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Qwen3 4b Grpo Merged F32 Final
llama_model_loader: - kv   3:                           general.finetune str              = grpo-merged-final
llama_model_loader: - kv   4:                           general.basename str              = qwen3
llama_model_loader: - kv  

# Step 5: Moving Final File to Drive

In [18]:
!cp /content/qwen_q4_k_m.gguf {OUTPUT_GGUF_DIR}/qwen_q4_k_m.gguf

In [19]:
print(f"\n>>> DONE. The GGUF file is at: {OUTPUT_GGUF_DIR}/qwen_q4_k_m.gguf")


>>> DONE. The GGUF file is at: /content/drive/MyDrive/grpo-verified-reasoner/models/qwen3-4b-grpo-f32-gguf/qwen_q4_k_m.gguf


# Step 6: Pushing to HF Hub

In [4]:
hf_token = userdata.get('HF_TOKEN')

In [11]:
local_model_path = "models/qwen3-4b-grpo-merged-f32-final"
local_gguf_path = "models/qwen3-4b-grpo-f32-gguf/qwen_q4_k_m.gguf"

In [6]:
model, tokenizer = FastLanguageModel.from_pretrained(
    local_model_path,
    dtype=torch.float16, # Downcast F32 -> F16 for bandwidth/storage efficiency
    load_in_4bit=False,
)

==((====))==  Unsloth 2026.1.2: Fast Qwen3 patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
repo_id_main = "samyakshrestha/qwen3-4b-grpo-fp16"
model.push_to_hub(repo_id_main, token=hf_token)
tokenizer.push_to_hub(repo_id_main, token=hf_token)

README.md:   0%|          | 0.00/570 [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0002-of-00002.safetensors:   0%|          |  524kB / 3.08GB            

  ...0001-of-00002.safetensors:   1%|          | 37.6MB / 4.97GB            

Saved model to https://huggingface.co/samyakshrestha/qwen3-4b-grpo-fp16


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mp34s36wun/tokenizer.json:  98%|#########8| 11.2MB / 11.4MB            

In [8]:
repo_id_gguf = "samyakshrestha/qwen3-4b-grpo-gguf"
api = HfApi(token=hf_token)

In [9]:
create_repo(repo_id_gguf, repo_type="model", exist_ok=True, token=hf_token)

RepoUrl('https://huggingface.co/samyakshrestha/qwen3-4b-grpo-gguf', endpoint='https://huggingface.co', repo_type='model', repo_id='samyakshrestha/qwen3-4b-grpo-gguf')

In [14]:
api.upload_file(
    path_or_fileobj=local_gguf_path,
    path_in_repo="qwen3-4b-grpo-q4_k_m.gguf", # Clean filename for the repo
    repo_id=repo_id_gguf,
    repo_type="model",
)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...f32-gguf/qwen_q4_k_m.gguf:   1%|1         | 25.2MB / 2.50GB            

CommitInfo(commit_url='https://huggingface.co/samyakshrestha/qwen3-4b-grpo-gguf/commit/9af575c0f27eed2615912c46398dec8453a5f955', commit_message='Upload qwen3-4b-grpo-q4_k_m.gguf with huggingface_hub', commit_description='', oid='9af575c0f27eed2615912c46398dec8453a5f955', pr_url=None, repo_url=RepoUrl('https://huggingface.co/samyakshrestha/qwen3-4b-grpo-gguf', endpoint='https://huggingface.co', repo_type='model', repo_id='samyakshrestha/qwen3-4b-grpo-gguf'), pr_revision=None, pr_num=None)

# Step 7: Fixing Metadata

In [1]:
import nbformat
import os

In [2]:
from google.colab import drive, files
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# List the notebook directory to confirm the file exists
os.listdir("/content/drive/MyDrive/grpo-verified-reasoner/notebooks")

In [ ]:
notebook_path = "/content/drive/MyDrive/grpo-verified-reasoner/notebooks/10_GGUF_conversion.ipynb"

with open(notebook_path, "r") as f:
    nb = nbformat.read(f, as_version=4)

if "widgets" in nb.metadata:
    del nb.metadata["widgets"]

with open(notebook_path, "w") as f:
    nbformat.write(nb, f)

print("Notebook fixed and saved successfully!")